# Assignment: k-NN, regression) trong bài toán dự đoán giá bất động sản

Tổng quan: Ở bài tập này chúng ta áp dụng một số phương pháp hồi quy tuyến tính, hồi quy k-NN cho dự đoán giá bất động sản Montreal, Houston

## Câu hỏi 1: khảo sát tập dữ liệu đơn gian Iris data set
xem có bao nhiêu bản ghi, bao nhiêu thuộc tính, xem một mẫu đầu tiên, bỏ những cột dư thừa, ...

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

In [2]:
iris=pd.read_csv('data/iris.csv')

In [1]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

## Câu hỏi 2: dự đoán giá bất động sản tỉnh Montreal

Mục tiêu đưa ra dự báo giá nhà dựa trên các thông tin thông tin quan trọng về nhà, dựa vào các mô hình hồi quy tuyến tính, hồi quy tuyến tính Ridge, hồi quy Laso, và hồi quy k lân cận gần nhất. Đánh giá trung bình sai số tuyệt đối của các mô hình trên dữ liệu kiểm tra.

## Dữ liệu

In [78]:
import pandas as pd
data=pd.read_csv("data/final_dataDec.csv")

In [79]:
data.head()

,not_sold,2014,2013,2012,2011,2010,2009,2008,2007,2006,...,emergency_shelters,emergency_water,Facilities,fire_stations,Cultural,Monuments,police_stations,Vacant,Free_Parking,askprice
0,1,0,0,0,0,0,0,0,0,0,...,37,274,210,43,71,43,25,43,52,166000
1,1,0,0,0,0,0,0,0,0,0,...,19,120,1,15,24,5,7,0,4,349000
2,1,0,0,0,0,0,0,0,0,0,...,27,191,155,28,54,32,18,19,43,224900
3,1,0,0,0,0,0,0,0,0,0,...,10,82,0,13,10,2,5,0,1,585000
4,1,0,0,0,0,0,0,0,0,0,...,30,238,216,37,66,42,20,43,52,312900


In [80]:
features=data.columns.values

In [81]:
data.shape

(9717, 40)

## Xử lý dữ liệu với các giá trị missing
- Loại bỏ các bản ghi có giá trị missing
- Khôi phục giá trị mising bằng giá trị trung bình

In [82]:
len(features)

40

In [83]:
features[0] # Thuộc tính not_sold, 1 chưa bán, 0 đã bán

'not_sold'

In [84]:
features[1:14]# Các thuộc tính bán năm 2002-> 2014: 1 năm bán

array(['2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007',
       '2006', '2005', '2004', '2003', '2002'], dtype=object)

In [85]:
features[14:16]# Số giường, năm xây dựng

array(['num_bed', 'year_built'], dtype=object)

In [86]:
features[16:18]# Tọa độ

array(['longitude', 'latitude'], dtype=object)

In [87]:
features[18:21]# Số phòng, số phòng tắm, diện tích

array(['num_room', 'num_bath', 'living_area'], dtype=object)

In [88]:
features[21:26]# Tính chất bất động sản

array(['house', 'plex', 'chalet', 'loft', 'condo'], dtype=object)

In [89]:
features[26:30]

array(['num_parking', 'accessible_buildings', 'family_quality',
       'art_expos'], dtype=object)

In [90]:
features[30:35]

array(['emergency_shelters', 'emergency_water', 'Facilities',
       'fire_stations', 'Cultural'], dtype=object)

In [91]:
features[35:39]

array(['Monuments', 'police_stations', 'Vacant', 'Free_Parking'],
      dtype=object)

In [92]:
features[39] # Giá tài sản

'askprice'

Các biến phụ thuộc **39**, biến dự báo **1** 

In [93]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso
#from sklearn.select_model import cross_validation
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import normalize

In [94]:
def loadData(filename):
	data = np.genfromtxt(filename, delimiter=',', dtype=float, skip_header=1)
	return data

In [95]:
data=loadData("data/final_dataDec.csv")

In [96]:
def remove_missing_data(pX_train, feature_to_impute):
    X_train =  np.copy(pX_train)
    for i in range(X_train.shape[0]-1, 0, -1):
        for j in range(0, X_train.shape[1], 1):
            if feature_to_impute[j] != 0 and X_train[i, j] == 0:
                X_train = np.delete(X_train, i, 0)
                break
    return X_train

In [97]:
impute = np.array([0] * len(data[0]))
impute[14] = 2 # num_bed
impute[15] = 2 # year_built
impute[18] = 2 # num_room
impute[19] = 2 # num_bath
impute[20] = 1 # living_space

In [98]:
data_removal=remove_missing_data(data,impute)

In [99]:
data_removal.shape

(2289, 40)

In [100]:
def mean_imputation_pure(pX_train, feature_to_impute):
	X_train =  np.copy(pX_train)
	for i in range(0, len(feature_to_impute)):
		if feature_to_impute[i] == 0:
			continue
		non_zeros = 0
		for j in range(0, X_train.shape[0]):
			if X_train[j, i] != 0:
				non_zeros += 1
		mean = np.sum(X_train[:, i])/float(non_zeros)
		for j in range(0, X_train.shape[0]):
			if X_train[j, i] == 0:
				X_train[j, i] = mean
	return X_train

In [101]:
data_imputation=mean_imputation_pure(data,impute)

In [102]:
data_imputation.shape

(9717, 40)

### Dự báo giá nhà với dữ liệu data_removal (bỏ các bản ghi lỗi)
- Dữ liệu data_removal
- Chia dữ liệu thành dữ liệu huấn luyện (70%) và dữ liệu kiểm tra (30%)

In [103]:
xrm=data_removal[:,:39]## Biến độc lập
yrm=data_removal[:,39] # Biến phụ thuộc

In [104]:
from sklearn.model_selection import train_test_split
xrm_train, xrm_test, yrm_train, yrm_test = train_test_split(xrm, yrm,test_size = 0.3, random_state=42)

** Hồi quy tuyến tính Ridge**

Tính trung bình sai số tuyệt đối trên dữ liệu kiểm tra, tìm lambda tốt nhất, vẽ biểu diến sai số theo lambda

In [2]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

** Hồi quy tuyến tính Lasso**

Tính trung bình sai số tuyệt đối trên dữ liệu kiểm tra, tìm lambda tốt nhất, vẽ biểu diến sai số theo lambda

In [4]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

### Dự báo giá nhà với dữ liệu imputation (làm đầy các bản ghi bị lỗi)
- Dữ liệu data_imputation
- Chia dữ liệu thành dữ liệu huấn luyện (70%) và dữ liệu kiểm tra (30%)
- Thực hiện các câu hỏi như trên

** Hồi quy tuyến tính **

In [5]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

** Hồi quy tuyến tính Ridge**

In [6]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

** Hồi quy tuyến tính Lasso**

In [7]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####

**Hồi quy phi tuyến K lân cận gần nhất**

In [8]:
#### YOUR CODE HERE ####

#### END YOUR CODE #####